#  Adversarial Attack

We assume that you have read the [Adversarial Attacks Tutorial](./adversarial_attacks_tutorial.ipynb) carefully and run that notebook from scratch. 

In this notebook, you are required to process adversarial attacks for a small subset of [ImageNet Dataset](http://www.image-net.org/). We prepared 100 images from different categories (in `./input_dir/`), and the labels are encoded in `./input_dir/clean_image.list`.

For evaluation, each adversarial image generated by the attack model will be fed to an evaluation model, and we will calculate the successful rate of adversarial attacks. **The adversarial images that can fool the evaluation model and also the perturbations are less than *Max_Distance* will be considered as a success**, where the perturbations are measured by the L2 distance between the adversarial image and original image.

There are three tasks:
- **White-box attack**: the adversarial examples are crafted for the pretrained **MobileNetV2** model, and evaluated on the same **MobileNetV2** model.
- **Black-box attack**: the adversarial examples are crafted for the pretrained **MobileNetV2** model, but evaluated on the **MobileNet** model, which is different from MobileNetV2.
- **Black-box attack (after submission)**: you are required to submit the generated adversarial examples at last, and we will evaluate your adversarial examples on another model, which is invisible for you.

### Goal

We provide a simple FGSM example here, and you are required to implement your own attack methods to **achieve the attack successful rate as high as possible** (for all three tasks).

At last, you are required to submit this jupyter notebook and the generated adversarial images.
The final grade will be scored according to the **white-box successful rate**, **black-box successful rate**, **white-box (after submission) successful rate**.

In [66]:
# ! pip3 install cython
# ! pip3 install tensornets
# ! pip3 install numpy==1.16.1

In [67]:
import sys,os
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
from time import perf_counter
from utils import *
import tensornets as nets

## Load Images
We provided 100 images from different categories in `./input_dir/`, and the labels are encoded in `./input_dir/clean_image.list`.

In [68]:
images = []
with open('./input_dir/clean_image.list', 'r') as f:
    img_lines = f.readlines()
    for img_line in img_lines:
        imgname, label = img_line.strip('\n').split(' ')
        images.append((imgname, int(label)))

## Image Processing

Each input image must be preprocessed before fed into the models, such as normalization(subtracting the mean and then dividing by the standard deviation). In addition, each generated adversarial image must be reversely processed.
Note that different pretrained models in Tensorflow require different preprocessing.
We provided several `preprocess` and `reverse_preprocess` function for different deep networks in `./utils.py`.

By default, the two functions are designed for mobilenet models.
```python
preprocess(image, model="mobilenet")
reverse_preprocess(image, model="mobilenet")
```

If you want to change to other models, see `./utils.py` for more details.

We have downloaded several popular pretrained models, you can adopt these models as the attacked model.
## Pretrained Models in tensornets (nets)
    'DenseNet121', 'DenseNet169', 'DenseNet201', 
    'Inception1', 'Inception2', 'Inception3', 'Inception4', 'InceptionResNet2',
    'MobileNet25', 'MobileNet50', 'MobileNet75', MobileNet100', 
    'MobileNet35v2', 'MobileNet50v2', 'MobileNet75v2', 'MobileNet100v2', 'MobileNet130v2', 'MobileNet140v2', 
    'NASNetAlarge', 'NASNetAmobile', 'PNASNetlarge',
    'ResNet50', 'ResNet101', 'ResNet152', 'ResNet50v2', 'ResNet101v2', 'ResNet152v2', 'ResNet200v2', 
    'ResNeXt50c32', 'ResNeXt101c32', 'ResNeXt101c64', 'WideResNet50',
    'VGG16', 'VGG19', 
    'SqueezeNet'.

## Define the Attack Method

### TODO: implement your own attack methods.

###  Tips:
- We provide the simple FGSM attack method as an example here. You can try other attack methods (learned in this course), such as the iterative methods.
- For black-box attack, we adopt the `MobileNetV2` as the attacked model, and the generated adversarial images may failed in `MobileNet` (which indicates poor transferability). You can try other attacked models (except `MobileNet`) or model ensemble.

In [69]:
class Attack:
    def __init__(self, input_image):
        self.input_image = input_image
        
        # loss function
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
        
        # TODO: you may change your target model.
        # load the model which will be attacked
        self.attacked_model = nets.MobileNet50v2(input_image, reuse=tf.AUTO_REUSE)
        
    def generate_adversarial_example(self, input_label):
#         input_image = self.input_image
#         prediction = self.attacked_model
#         loss = self.loss_object(input_label, prediction)
#
#         # TODO: implement your own attack methods.
#         # Get the gradients of the loss w.r.t to the input image.
#         gradient = tf.gradients(loss, input_image)
        
#         # Get the sign of the gradients to create the perturbation (FGSM)
#         signed_grad = tf.sign(gradient)[0]
#         # Epsilon in FGSM, you can try another value.
#         eps = 0.05
#         adv_image = input_image + eps * signed_grad
        
#         # Clip the generated image between -1 and 1. Note that different pretrained models require different ranges.
#         adv_image = tf.clip_by_value(adv_image, -1, 1)
        
#         # END TODO
        input_image = self.input_image
        
        adv_image = input_image
        g = 0.0
        u = 1.0
        eps = 0.001
        
        for t in range(1,16):
            prediction = nets.MobileNet50v2(adv_image, reuse=tf.AUTO_REUSE)
            loss = self.loss_object(input_label, prediction)
            gradient = tf.gradients(loss, adv_image)
            g = u*g + gradient[0]/tf.norm(gradient,ord=1)
            adv_image = tf.clip_by_value(adv_image+eps/t*tf.sign(g)[0], -1, 1)
            
        return adv_image

# Evaluation
Define the evaluation functions for both white-box and black-box attack.
**You are not allowed to modify these codes.**

- For white-box attack, the adversarial images are evaluated on the `MobileNetv2` model.
- For black-box attack, the adversarial images are evaluated on the `MobileNet` model. Therefore, you can not use the same `MobileNet` model as the attacked model.

The `Max_Distance` equals to 5.0 here.

In [70]:
Max_Distance = 5.0

class WhiteBox_Evaluation:
    def __init__(self, adv_image):
        self.adv_image = adv_image
        self.eval_model = nets.MobileNet50v2(adv_image, reuse=tf.AUTO_REUSE)
        
    def get_adv_label(self):
        adv_probs  = self.eval_model
        adv_label = tf.argmax(adv_probs,1)
        return adv_label
    
class BlackBox_Evaluation:
    def __init__(self, adv_image):
        self.adv_image = adv_image
        self.eval_model = nets.MobileNet50(adv_image, reuse=tf.AUTO_REUSE)
        
    def get_adv_label(self):
        adv_probs  = self.eval_model
        adv_label = tf.argmax(adv_probs,1)
        return adv_label
    
# init the attacker
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
config.allow_soft_placement = True
sess = tf.Session(config=config)

# load and preprocess image
input_path = tf.placeholder(dtype=tf.string)
input_label = tf.placeholder(shape=None, dtype=tf.int32)
image_raw = tf.io.read_file(input_path)
image = tf.image.decode_jpeg(image_raw, channels=3)
image = image[None, ...]

input_image = preprocess(image)
attacker = Attack(input_image)

# generate adversarial example
adv_image_t = attacker.generate_adversarial_example(input_label)
eval_model_white = WhiteBox_Evaluation(adv_image_t)
eval_model_black = BlackBox_Evaluation(adv_image_t)

# measured by L2 distance
distance_t = tf.math.reduce_euclidean_norm(input_image - adv_image_t)

adv_label_white_t = eval_model_white.get_adv_label()
adv_label_black_t = eval_model_black.get_adv_label()

saved_image_t = reverse_preprocess(adv_image_t)[0]

sess.run(tf.global_variables_initializer())
_ = sess.run([attacker.attacked_model.pretrained(), eval_model_white.eval_model.pretrained(), eval_model_black.eval_model.pretrained()])


# White-Box Attack Evaluation

In [71]:
success_cnt = 0

for idx, (imgname, label) in enumerate(images):
    imgpath = './input_dir/' + imgname
    run_list = [adv_image_t, distance_t, adv_label_white_t, saved_image_t]
    feed_dict = {input_path: imgpath, input_label: label}
    
    adv_image, distance, adv_label, saved_image = sess.run(run_list, feed_dict)
    adv_label = adv_label[0]
    
    # if the adversarial image can successfully fool the attacked model, and the perturbations are less than Max_Distance
    if distance <= Max_Distance:
        success_cnt += 1 if adv_label != label else 0
    
    print('{}: clean_label={:3d} adv_label={:3d} distance={:.2f}'.format(imgname,label,adv_label,distance))
    
    # save the generated images to './output_dir'
    saved_image = tf.image.encode_png(saved_image)
    write_ops = tf.io.write_file('./output_dir/' + imgname, saved_image)
    sess.run(write_ops)

print()
print('White-box attack successful rate: {}%'.format(success_cnt))

n02708093.JPEG: clean_label=409 adv_label=592 distance=0.97
n03000134.JPEG: clean_label=489 adv_label=352 distance=1.15
n03384352.JPEG: clean_label=561 adv_label=495 distance=1.07
n03777754.JPEG: clean_label=662 adv_label=882 distance=1.09
n03721384.JPEG: clean_label=642 adv_label=696 distance=1.11
n03424325.JPEG: clean_label=570 adv_label=691 distance=1.09
n03673027.JPEG: clean_label=628 adv_label=833 distance=1.09
n02229544.JPEG: clean_label=312 adv_label=456 distance=1.07
n07695742.JPEG: clean_label=932 adv_label=925 distance=1.11
n02018207.JPEG: clean_label=137 adv_label=137 distance=1.13
n02107908.JPEG: clean_label=240 adv_label=178 distance=1.07
n04026417.JPEG: clean_label=748 adv_label=643 distance=1.11
n02444819.JPEG: clean_label=360 adv_label=150 distance=1.13
n02259212.JPEG: clean_label=317 adv_label=462 distance=1.10
n02480495.JPEG: clean_label=365 adv_label=341 distance=1.11
n02095889.JPEG: clean_label=190 adv_label=190 distance=1.13
n03216828.JPEG: clean_label=536 adv_labe

# Black-Box Attack Evaluation

In [72]:
success_cnt = 0

for idx, (imgname, label) in enumerate(images):
    imgpath = './input_dir/' + imgname
    run_list = [adv_image_t, distance_t, adv_label_black_t, saved_image_t]
    feed_dict = {input_path: imgpath, input_label: label}
    
    adv_image, distance, adv_label, saved_image = sess.run(run_list, feed_dict)
    adv_label = adv_label[0]
    
    # if the adversarial image can successfully fool the attacked model, and the perturbations are less than Max_Distance
    if distance <= Max_Distance:
        success_cnt += 1 if adv_label != label else 0
    
    print('{}: clean_label={:3d} adv_label={:3d} distance={:.2f}'.format(imgname,label,adv_label,distance))
    
    # save the generated images to './output_dir'
    saved_image = tf.image.encode_png(saved_image)
    write_ops = tf.io.write_file('./output_dir/' + imgname, saved_image)
    sess.run(write_ops)

print()
print('Black-box attack successful rate: {}%'.format(success_cnt))

n02708093.JPEG: clean_label=409 adv_label=409 distance=0.97
n03000134.JPEG: clean_label=489 adv_label=489 distance=1.15
n03384352.JPEG: clean_label=561 adv_label=561 distance=1.07
n03777754.JPEG: clean_label=662 adv_label=662 distance=1.09
n03721384.JPEG: clean_label=642 adv_label=642 distance=1.11
n03424325.JPEG: clean_label=570 adv_label=570 distance=1.09
n03673027.JPEG: clean_label=628 adv_label=628 distance=1.09
n02229544.JPEG: clean_label=312 adv_label=872 distance=1.07
n07695742.JPEG: clean_label=932 adv_label=925 distance=1.11
n02018207.JPEG: clean_label=137 adv_label=137 distance=1.13
n02107908.JPEG: clean_label=240 adv_label=241 distance=1.07
n04026417.JPEG: clean_label=748 adv_label=748 distance=1.11
n02444819.JPEG: clean_label=360 adv_label=360 distance=1.13
n02259212.JPEG: clean_label=317 adv_label=317 distance=1.10
n02480495.JPEG: clean_label=365 adv_label=365 distance=1.11
n02095889.JPEG: clean_label=190 adv_label=190 distance=1.13
n03216828.JPEG: clean_label=536 adv_labe